In [8]:
"""
 Second way --- using the "webdriver-manager"
 we will be able to always download the latest version of the "chromedriver" 
 using our Google Chrome version as a reference.
"""

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

#allow us to navigate trough any page and to open a html archive in our pc.
driver.get(r"https://www.zolo.ca/")
#id = "13228072"

from selenium.webdriver.common.by import By

#driver.get(r"/Users/aniziocp/Desktop/Github/toronto-real-state-scraping/zolo.html")
title = driver.find_element(By.XPATH, '//*[@id="listing_container"]/section[1]/h1')


print(title)

#best order of elements -> ID > XPATH > CLASS NAME

<selenium.webdriver.remote.webelement.WebElement (session="9644a03448a151a9df4f89f70b52d406", element="a3652a54-8e96-4e30-89e0-4500acc29a39")>


In [7]:
title

<selenium.webdriver.remote.webelement.WebElement (session="11046be15b7d1b088887edb16fdc18f8", element="26a7770f-4126-4a0e-8acf-41a62226abc2")>